In [1]:
import json
import numpy as np
import sys 
from matplotlib import pyplot as plt


In [25]:
tasks = [
    "sentiment2/glue-sst2", 
    "sentiment5/SetFit/sst5-None",
    "sentiment2-yelp/yelp_polarity-None",
    "sentiment5-yelp/yelp_review_full-None", 
    "sentiment2-amazon/amazon_polarity-None",
    "sentiment2-mr/rotten_tomatoes-None",
    "sentiment2-cr/SetFit/CR-None",
    "tweet_eval-sentiment/tweet_eval-None",
    "finance_sentiment3/financial_phrasebank-None",
    "poemsentiment2/poem_sentiment-None",
    "agnews/ag_news-None",
    "dbpedia/dbpedia_14-None",
    "hate_speech18/hate_speech18-None",
    "tweet_eval-hate/tweet_eval-None",
    "ethos-national_origin/ethos-national_origin-None2",
    "ethos-sexual_orientation/ethos-sexual_orientation-None2",
    "ethos-race/ethos-race-None2",
    "ethos-religion/ethos-religion-None2",
    "emotion6/dair-ai/emotion-None",
    ]
task_names = [
    "SST2",
    "SST5",
    "Yelp2",
    "Yelp5",
    "Amazon2",
    "MR",
    "CR",
    "Tweet3",
    "FP",
    "PS", 
    "AGNews",
    "DBPedia", 
    "HS18",
    "Tweet",
    "Ethos (NO)",
    "Ethos (SO)",
    "Ethos (Race)",
    "Ethos (Religion)", 
    "Emotions"
]
models = [
    "gpt2", 
    "gpt2-medium", 
    "gpt2-large", 
    "gpt2-xl", 
    "facebook/opt-1.3b", 
    "facebook/opt-2.7b", 
    "EleutherAI/pythia-1.4b", 
    "EleutherAI/pythia-2.8b", 
    "EleutherAI/pythia-6.7b",
    "EleutherAI/pythia-12b",
    "EleutherAI/gpt-j-6b",
    "huggyllama/llama-7b",
    "huggyllama/llama-13b",
    "meta-llama/Llama-2-7b-hf",
    "meta-llama/Llama-2-13b-hf",
    ]

approaches = ["channel-simple", "direct-simple", "direct-context", "direct-instruct"]
has_pp = [False, True, True, True]
num_tasks = len(tasks)

19


In [28]:
def compute_mean_std(lines_main, lines_text, prefix, task, metric, arithmetic, geometric, harmonic, metric2, arithmetic2, geometric2, harmonic2):

    x, yarith_mean, yarith_std, F1arith_mean, F1arith_std = get_stats(lines_main, prefix, arithmetic, metric)
    x, ygeo_mean, ygeo_std, F1geo_mean, F1geo_std = get_stats(lines_main, prefix, geometric, metric)
    x, yharm_mean, yharm_std, F1harm_mean, F1harm_std = get_stats(lines_main, prefix, harmonic, metric)

    return_object = [np.array(x), yarith_mean, ygeo_mean, yharm_mean, yarith_std, ygeo_std, yharm_std]
    return_objectf1 = [np.array(x), F1arith_mean, F1geo_mean, F1harm_mean, F1arith_std, F1geo_std, F1harm_std]

    # if plot:
    #     plt.errorbar(x, yarith_mean, yarith_std, color="blue", label="Arithmetic")
    #     plt.errorbar(x, ygeo_mean, ygeo_std, color="red", label="Geometric")
    #     plt.errorbar(x, yharm_mean, yharm_std, color="green", label="Harmonic")

    # if x is not None:
    #     print(f'{task}, {prefix} & {(F1arith_mean[-2] * 100):.1f}$_{{({(F1arith_std[-2]*100):.2f})}}$')
    # else:
    #     print(f'{task}, {prefix} & NA')

    #print(lines_text)
    x, yarith_mean, yarith_std, F1arith_mean, F1arith_std = get_stats(lines_text, prefix, arithmetic2, metric2)
    x, ygeo_mean, ygeo_std, F1geo_mean, F1geo_std = get_stats(lines_text, prefix, geometric2, metric2)
    x, yharm_mean, yharm_std, F1harm_mean, F1harm_std = get_stats(lines_text, prefix, harmonic2, metric2)

    # if x is not None:
    #     print(f'{task}-text, {prefix} & {(yarith_mean[-2] * 100):.1f}$_{{({(yarith_std[-2]*100):.2f})}}$')
    # else:
    #     print(f'{task}-text, {prefix} & NA')

    return_object += [yarith_mean, ygeo_mean, yharm_mean, yarith_std, ygeo_std, yharm_std]
    return_objectf1 += [F1arith_mean, F1geo_mean, F1harm_mean, F1arith_std, F1geo_std, F1harm_std]

    return return_object, return_objectf1

    # if plot:
    #     plt.errorbar(x, yarith_mean, yarith_std, color="blue", label="Arithmetic (w/o domain)")
    #     plt.errorbar(x, ygeo_mean, ygeo_std, color="red", label="Geometric (w/o domain)")
    #     plt.errorbar(x, yharm_mean, yharm_std, color="green", label="Harmonic (w/o domain)")

    #     plt.legend(loc="best")
    #     plt.savefig(save_figfile)
    #     plt.clf()
    #input()

In [29]:
def precision_recall_fscore(confusion_matrix, i):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for j in range(len(confusion_matrix)):
        if (i == j):
            TP += confusion_matrix[i][j]
            tmp = np.delete(confusion_matrix, i, 0)
            tmp = np.delete(tmp, j, 1)

            TN += np.sum(tmp)
        else:
            if (confusion_matrix[i][j] != 0):

                FN += confusion_matrix[i][j]
            if (confusion_matrix[j][i] != 0):

                FP += confusion_matrix[j][i]

    recall = TP / (FN + TP + 1e-10)
    precision = TP / (TP + FP + 1e-10)
    f1_score = 2 * 1/(1/(recall + 1e-10) + 1/(precision + 1e-10))

    return precision, recall, f1_score

def f1_score(confusion_matrix, macro="True"):
    Ps, Rs, F1s = [], [], []
    for i in range(len(confusion_matrix)):
        p, r, f1 = precision_recall_fscore(confusion_matrix, i)
        Ps.append(p)
        Rs.append(r)
        F1s.append(f1)
    
    return Ps, Rs, F1s, np.mean(F1s)

def get_stats(lines, prefix, suffix, metric):
    y = []
    F1 = []
    x = None
    c = 0
    for line in lines:
        try:
            items = json.loads(line)
        except Exception as e:
            print(line)
            print(e)
            input(line)
            raise ValueError
        if x is None:
            x = items['k']
        y.append(items[prefix+f"{metric}"+suffix])

        tmp = items[prefix+"confusion_matrix"+suffix]
        new_temp = []
        for item in tmp:
            temp_item = item.replace("\n", "")
            while "[ " in temp_item:
                temp_item = temp_item.replace("[ ", "[")
            while "  " in temp_item:
                temp_item = temp_item.replace("  ", " ")
            new_temp.append(eval(temp_item.replace(" ", ",")))
        tmp = new_temp
        # tmp = [eval(item.replace("\n", "").replace("[ ", "[").replace("[ ", "[").replace("  ", " ").replace(" ", ",")) for item in tmp]
        F1.append([f1_score(cm)[-1] for cm in tmp])

    if len(F1) > 0:
        y_mean  = np.mean(y, axis=0)
        y_std = np.std(y, axis=0)
        F1_mean  = np.mean(F1, axis=0)
        F1_std = np.std(F1, axis=0)

        return x[:10], y_mean[:10], y_std[:10], F1_mean[:10], F1_std[:10]
    else:
        print(lines)
        input()
        return None, None, None, None, None

In [31]:
results = {}
import os
prefix_channel=""
prefix_direct="direct_"
prefix_directplusplus="direct++_"
notdone = {}
for model in models:
    notdone[model] = {}
    # if "pythia" in model:
    #     break
    print("model: "+model)
    results[model] = {}
    for i, approach in enumerate(approaches):
        notdone[model][approach] = {'main': [], 'text': []}
        if "direct" in approach:
            results[model][approach] = {"direct": {"mean": [0 for _ in range(13)], "mean_f1": [0 for _ in range(13)]}, 
                                        "direct++": {"mean": [0 for _ in range(13)], "mean_f1": [0 for _ in range(13)]}}
        else:
            results[model][approach] = {"channel": {"mean": [0 for _ in range(13)], "mean_f1": [0 for _ in range(13)]}}

        for task in tasks:
            # print(task)
            # for key in results[model][approach]:
            #     results[model][approach][key][task] = {}
            filename_main = f"./results/0923/{approach}/{task}/{model}/results.jsonl"
            filename_text = f"./results/0923-ablations/{approach}/{task}/{model}/results.jsonl"            
            
            try:
                lines_main = open(filename_main).readlines()
            except:
                notdone[model][approach]['main'].append(task)
                print(f"{filename_main} does not exist. Skipping")
                continue

            if len(lines_main) == 0:
                notdone[model][approach]['main'].append(task)
                print(f"{filename_main} is empty. Skipping")
                continue
            # print(filename_main, filename_text)
            try:
                lines_text = open(filename_text).readlines()
            except:
                lines_text = open(filename_main).readlines()
                notdone[model][approach]['text'].append(task)
                # print(f"{filename_text} does not exist. So using the same file in both. BEWARE")
                # continue

            if len(lines_text) == 0:
                notdone[model][approach]['text'].append(task)
                lines_text = lines_main
                # print(f"{filename_text} is empty. So using the same file in both. BEWARE")

            # print(filename_main)
            try:
                metric_name = json.loads(lines_main[0])['metric_name']
                metric = "metric"
                arithmetic = "_arithmetic"
                geometric = "_geometric"
                harmonic = "_harmonic"
            except:
                arithmetic = "_logsumexp"
                geometric = "_average"
                harmonic = "_vote"
                metric_name = "accuracy"
                metric = "accuracy"

            try:
                metric_name2 = json.loads(lines_text[0])['metric_name']
                metric2 = "metric"
                arithmetic2 = "_arithmetic"
                geometric2 = "_geometric"
                harmonic2 = "_harmonic"
            except:
                arithmetic2 = "_logsumexp"
                geometric2 = "_average"
                harmonic2 = "_vote"
                metric_name2 = "accuracy"
                metric2 = "accuracy"

            if "channel" in approach:
                # os.makedirs(f'./results/0923/plots/{model.replace("/", "-")}/{approach}/', exist_ok=True)
                # save_figfile = f'./results/0923/plots/{model.replace("/", "-")}/{approach}/{task.replace("/", "-")}-{metric}.png'
                acc, f1 = compute_mean_std(lines_main, lines_text, "", task, metric, arithmetic, geometric, harmonic, metric2, arithmetic2, geometric2, harmonic2)
                results[model][approach]["channel"][task] = acc
                results[model][approach]["channel"][task+'_f1'] = f1
            else:
                # os.makedirs(f'./results/0923/plots/{model.replace("/", "-")}/{approach}/', exist_ok=True)
                # save_figfile = f'./results/0923/plots/{model.replace("/", "-")}/{approach}/{task.replace("/", "-")}.png'
                acc, f1 = compute_mean_std(lines_main, lines_text, "direct_", task, metric, arithmetic, geometric, harmonic, metric2, arithmetic2, geometric2, harmonic2)
                results[model][approach]["direct"][task] = acc
                results[model][approach]["direct"][task+"_f1"] = f1

                # os.makedirs(f'./results/0923/plots/{model.replace("/", "-")}/{approach}++/', exist_ok=True)
                # save_figfile = f'./results/0923/plots/{model.replace("/", "-")}/{approach}++/{task.replace("/", "-")}.png'
                acc, f1 = compute_mean_std(lines_main, lines_text, "direct++_", task, metric, arithmetic, geometric, harmonic, metric2, arithmetic2, geometric2, harmonic2)
                results[model][approach]["direct++"][task] = acc
                results[model][approach]["direct++"][task+"_f1"] = f1

            # input()
        for task in tasks:
            for j in range(13):
                # print(j)
                try:
                    if "channel" in approach:
                        results[model][approach]["channel"]["mean"][j] = results[model][approach]["channel"]["mean"][j] + results[model][approach]["channel"][task][j]
                        results[model][approach]["channel"]["mean_f1"][j] = results[model][approach]["channel"]["mean_f1"][j] + results[model][approach]["channel"][task+"_f1"][j]
                    else:
                        #if results[model][approach][approach][task][j] is not None and results[model][approach][approach+"++"][task][j] is not None:
                        results[model][approach]["direct"]["mean"][j] = results[model][approach]["direct"]["mean"][j] + results[model][approach]["direct"][task][j]
                        results[model][approach]["direct"]["mean_f1"][j] = results[model][approach]["direct"]["mean_f1"][j] + results[model][approach]["direct"][task+"_f1"][j]

                        results[model][approach]["direct++"]["mean"][j] = results[model][approach]["direct++"]["mean"][j] + results[model][approach]["direct++"][task][j]
                        results[model][approach]["direct++"]["mean_f1"][j] = results[model][approach]["direct++"]["mean_f1"][j] + results[model][approach]["direct++"][task+"_f1"][j]
                        # c += 1
                except Exception as e:
                    print(f"something happend to {model} {task} item {j} {e}")
                    # print(e)
                    # input()
                    
                    # break
                    # continue
            # print(tasks)
        # break
    # break
# 

model: gpt2
model: gpt2-medium
model: gpt2-large
model: gpt2-xl
model: facebook/opt-1.3b
model: facebook/opt-2.7b
model: EleutherAI/pythia-1.4b
model: EleutherAI/pythia-2.8b
model: EleutherAI/pythia-6.7b
model: EleutherAI/pythia-12b
./results/0923/direct-simple/dbpedia/dbpedia_14-None/EleutherAI/pythia-12b/results.jsonl is empty. Skipping
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 0 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 1 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 2 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 3 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 4 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_14-None item 5 'dbpedia/dbpedia_14-None'
something happend to EleutherAI/pythia-12b dbpedia/dbpedia_

In [23]:
for model in models:
    print("model: "+model)
    
    for i, approach in enumerate(approaches):
        if "channel" in approach:
            lengths = [len(results[model][approach]["channel"])]
            keys = ["channel"]
        else:
            lengths = [len(results[model][approach]["direct"])]
            lengths += [len(results[model][approach]["direct++"])]
            keys = ["direct", "direct++"]
        os.makedirs(f'./results/0923/plots/iclr/{approach}', exist_ok=True)
        for key, length in zip(keys, lengths):
            save_figfile = f'./results/0923/plots/iclr/{approach}/{model.replace("/", "-")}_{approach}_{key}_all-tasks-mean-accuracy.png'
            # print(save_figfile)
            x = results[model][approach][key]["mean"][0]/length
            
            main_arith = results[model][approach][key]["mean"][1]/length
            main_geo = results[model][approach][key]["mean"][2]/length
            main_harm = results[model][approach][key]["mean"][3]/length

            main_arith_std = results[model][approach][key]["mean"][4]/length
            main_geo_std = results[model][approach][key]["mean"][5]/length
            main_harm_std = results[model][approach][key]["mean"][6]/length

            text_arith = results[model][approach][key]["mean"][7]/length
            text_geo = results[model][approach][key]["mean"][8]/length
            text_harm = results[model][approach][key]["mean"][9]/length

            text_arith_std = results[model][approach][key]["mean"][10]/length
            text_geo_std = results[model][approach][key]["mean"][11]/length
            text_harm_std = results[model][approach][key]["mean"][12]/length

            plt.plot(x, main_arith, "s-", color="blue", label="Arithmetic", )
            plt.fill_between(x, main_arith-main_arith_std, main_arith+main_arith_std, color="blue", edgecolor="none", alpha=0.1)

            plt.plot(x, main_geo, "s-", color="red", label="Geometic")
            plt.fill_between(x, main_geo-main_geo_std, main_geo+main_geo_std, color="red", edgecolor="none", alpha=0.1)

            plt.plot(x, main_harm, "s-", color="green", label="Harmonic")
            plt.fill_between(x, main_harm-main_harm_std, main_harm+main_harm_std, color="green", edgecolor="none", alpha=0.1)

            plt.plot(x, text_arith, "o-", color="blue", label="Arithmetic (no context)")
            plt.fill_between(x, text_arith-text_arith_std, text_arith+text_arith_std, color="blue", edgecolor="none", alpha=0.1)

            plt.plot(x, text_geo, "o-", color="red", label="Geometic (no context)")
            plt.fill_between(x, text_geo-text_geo_std, text_geo+text_geo_std, color="red", edgecolor="none", alpha=0.1)

            plt.plot(x, text_harm, "o-", color="green", label="Harmonic (no context)")
            plt.fill_between(x, text_harm-text_harm_std, text_harm+text_harm_std, color="green", edgecolor="none", alpha=0.1)

            
            # plt.errorbar(x, main_arith, main_arith_std, color="blue", label="Arithmetic")
            # plt.errorbar(x, main_geo, main_geo_std, color="red", label="Geometric")
            # plt.errorbar(x, main_harm, main_harm_std, color="green", label="Harmonic")

            # plt.errorbar(x, text_arith, text_arith_std, color="blue", marker="o", label="Arithmetic (w/o domain)")
            # plt.errorbar(x, text_geo, text_geo_std, color="red", marker="o", label="Geometric (w/o domain)")
            # plt.errorbar(x, text_harm, text_harm_std, color="green", marker="o", label="Harmonic (w/o domain)")
            plt.title(model)
            plt.legend(loc="best")
            plt.savefig(save_figfile)
            plt.clf()
            # plt.show()
        # input()
    # break
            # plt.clf()

            if "gpt-j" in model:
                for x, y, ye in zip(range(10), main_arith, main_arith_std):
                    print(x, y, ye)
                print()
                for x, y, ye in zip(range(10), text_arith, text_arith_std):
                    print(x, y, ye)



model: gpt2
./results/0923/plots/iclr/channel-simple/gpt2_channel-simple_channel_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-simple/gpt2_direct-simple_direct_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-simple/gpt2_direct-simple_direct++_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-context/gpt2_direct-context_direct_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-context/gpt2_direct-context_direct++_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-instruct/gpt2_direct-instruct_direct_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-instruct/gpt2_direct-instruct_direct++_all-tasks-mean-accuracy.png
model: gpt2-medium
./results/0923/plots/iclr/channel-simple/gpt2-medium_channel-simple_channel_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-simple/gpt2-medium_direct-simple_direct_all-tasks-mean-accuracy.png
./results/0923/plots/iclr/direct-simple/gpt2-medium_direct-simple_direct++_all-tasks-

<Figure size 640x480 with 0 Axes>

In [27]:
create_results_table("EleutherAI/gpt-j-6b", results, "f1")
# results["EleutherAI/gpt-j-6b"]['direct-simple']['direct']    

task direct-simple direct-context direct-instruct channel
SST2  & 69.5\std{1.2} & 65.9\std{0.9} & 43.6\std{1.0} & 74.5\std{1.2} & 75.7\std{1.3} & 72.3\std{0.7} & 80.0\std{1.0} & 87.1\std{0.6} & \textbf{91.7}\std{0.2}\\
SST5  & 18.6\std{0.7} & 24.2\std{0.9} & 25.1\std{0.2} & 26.2\std{0.7} & 29.8\std{0.6} & 35.6\std{0.2} & 34.8\std{0.8} & 36.0\std{1.2} & \textbf{40.9}\std{0.5}\\
Yelp2  & 68.1\std{1.0} & 64.2\std{1.1} & 76.7\std{0.3} & 70.5\std{0.5} & 70.2\std{0.6} & 75.9\std{0.5} & 76.1\std{0.8} & 85.4\std{0.7} & \textbf{89.9}\std{0.1}\\
Yelp5  & 24.0\std{1.1} & 26.2\std{0.7} & 25.6\std{0.3} & 30.5\std{1.1} & 28.5\std{1.4} & 32.0\std{0.3} & 35.5\std{0.8} & 38.0\std{0.7} & \textbf{42.0}\std{0.3}\\
Amazon2  & 61.0\std{1.7} & 61.0\std{1.7} & 58.2\std{0.5} & 72.7\std{1.2} & 72.7\std{1.2} & \textbf{78.7}\std{0.4} & 68.4\std{1.1} & 68.4\std{1.1} & 71.3\std{0.4}\\
MR  & 67.7\std{0.9} & 63.3\std{1.0} & 51.3\std{0.5} & 72.3\std{1.0} & 70.6\std{1.2} & 74.6\std{0.5} & 76.0\std{0.9} & 84.2\std{0.6} 

In [26]:
# "direct-context", "direct-instruct", 
print(len(tasks))
def create_results_table(model, results, metric="accuracy"):
    print("task", "direct-simple", "direct-context", "direct-instruct", "channel")
    for task, task_name in zip(tasks, task_names):
        print(f"{task_name} ", end="")
        items = []
        for approach in ["direct-simple", "channel-simple"]:
            key = task
            if metric == "f1":
                key += "_f1"
            try:
                if "channel" in approach:
                    result_array = [results[model][approach]['channel'][key]]
                else:
                    result_array = [results[model][approach]['direct'][key], results[model][approach]['direct++'][key]]
                
                for result in result_array:
                    # if metric == "accuracy":
                    # items = result[:7]

                    x = result[0]

                    arithmean = result[1]
                    arithstd = result[4]

                    geomean = result[2]
                    geostd = result[5]

                    harmmean = result[3]
                    harmstd = result[6]

                    nocontext_arithmean = result[7]
                    nocontext_arithstd = result[10]

                    nocontext_geomean = result[8]
                    nocontext_geostd = result[11]

                    nocontext_harmmean = result[9]
                    nocontext_harmstd = result[12]
                    
                    items += [(nocontext_arithmean[0], nocontext_arithstd[0]), (arithmean[0], arithstd[0]), (arithmean[-1], arithstd[-1])]

                    # print(f"& {nocontext_arithmean[0]*100:.1f}$_{{({nocontext_arithstd[0]*100:.1f})}}$ & {arithmean[0]*100:.1f}$_{{({arithstd[0]*100:.1f})}}$ & {arithmean[-1]*100:.1f}$_{{({arithstd[-1]*100:.1f})}}$", end="")
                    # print(f"& {nocontext_geomean[0]*100:.1f}$_{{({nocontext_geostd[0]*100:.1f})}}$ & {geomean[0]*100:.1f}$_{{({geostd[0]*100:.1f})}}$ & {geomean[-1]*100:.1f}$_{{({geostd[-1]*100:.1f})}}$", end="")
                    # print(f"& {nocontext_harmmean[0]*100:.1f}$_{{({nocontext_harmstd[0]*100:.1f})}}$ & {harmmean[0]*100:.1f}$_{{({harmstd[0]*100:.1f})}}$ & {harmmean[-1]*100:.1f}$_{{({harmstd[-1]*100:.1f})}}$", end="")
            except:
                if "channel" in approach:
                    items += [(0,0), (0,0), (0,0)]
                    # print(f"& & &", end=0)
                else:
                    items += [(0,0), (0,0), (0,0), (0,0), (0,0), (0,0)]
                    # print(f"& & & & & & ", end="")
        
        # print(len(items))
        max_mean = max(items, key=lambda x: x[0])
        for item in items:
            if item[0] == max_mean[0]:
                print(f" & \\textbf{{{item[0]*100:.1f}}}\std{{{item[1]*100:0.1f}}}", end="")
            else:
                print(f" & {item[0]*100:.1f}\std{{{item[1]*100:0.1f}}}", end="")
            
        print("\\\\")

        
                # else:
                    # items = result[:1] + result[7:]
   

19
